#  4. Construct Digital Twin Simulation for Operating Mobility Systems

### 4.1 간단한 예제로 알아보는 모빌리티 디지털 트윈 시뮬레이션 구현

- 상세한 시뮬레이션의 각 구성요소를 탐색하기 전에 대략적으로 어떻게 시뮬레이션이 구동되고 결과가 도출되는지 실습

#### 4.1.1 가상의 통행데이터 생성

In [1]:
import numpy as np
import itertools
import requests
import polyline
import json
import os

import random as rd
import pandas as pd

from shapely.geometry import Point

from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

import warnings 

warnings.filterwarnings('ignore')

##### [1] 필수 함수 정의

In [2]:
# 직선 거리 계산 함수
def calculate_straight_distance(lat1, lon1, lat2, lon2):
    # 지구 반경 (킬로미터 단위)
    km_constant = 3959* 1.609344
    # 위도와 경도를 라디안으로 변환
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    # 위도 및 경도 차이 계산
    dlat = lat2 - lat1 
    dlon = lon2 - lon1
    # Haversine 공식 계산
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    # 거리 계산 (킬로미터 단위)
    km = km_constant * c
    
    return km

In [3]:
# osrm 기반의 Route 생성 함수
# 입력으로 받은 출발지와 목적지 좌표를 이용하여 경로 정보를 가져오는 함수
def get_res(point):

   status = 'defined'

   # 요청을 재시도할 수 있도록 세션 객체 생성 및 설정
   session = requests.Session()
   retry = Retry(connect=3, backoff_factor=0.5)
   adapter = HTTPAdapter(max_retries=retry)
   session.mount('http://', adapter)
   session.mount('https://', adapter)

   #### url 생성 코드
   # 전체 경로 정보를 요청
   overview = '?overview=full'
   # lon, lat, lon, lat 형식의 출발지 목적지 좌표
   loc = f"{point[0]},{point[1]};{point[2]},{point[3]}"
   # 보행경로 url
   url = 'http://router.project-osrm.org/route/v1/foot/'
   # 경로 정보 요청
   r = session.get(url + loc + overview) 
   
   # 만약 경로가 안뜰 때 대체 결과 생성
   if r.status_code!= 200:
      
      status = 'undefined'
      
       # 직선 거리 계산
      distance = calculate_straight_distance(point[1], point[0], point[3], point[2]) * 1000
      
      # 경로 정보 생성 (출발지와 목적지 좌표만 포함)
      route = [[point[0], point[1]], [point[2], point[3]]]

      # 소요 시간 및 타임스탬프 계산 (가정: 보행 속도 10km/h)
      speed_km = 10#km
      speed = (speed_km * 1000/60)      
      duration = distance/speed
      
      timestamp = [0, duration]

      result = {'route': route, 'timestamp': timestamp, 'duration': duration, 'distance' : distance}
   
      return result, status
   
   # 경로 정보를 성공적으로 가져온 경우, JSON 응답을 반환
   res = r.json()   
   return res, status

In [4]:
# 경로를 가는데 걸리는 시간과 거리 추출 함수
def extract_duration_distance(res):
   # get_res함수에서 추출된 데이터에서 시간과 거리 뽑기
   
   duration = res['routes'][0]['duration']/(60)  # 분 단위로 변환
   distance = res['routes'][0]['distance']
   
   return duration, distance

# 경로 추출 함수
def extract_route(res):
   
    # get_res함수에서 추출된 데이터에서 경로 뽑기
    # 경로가 인코딩 되어 있기 때문에 아래 함수를 써서 디코딩해주어야지 위경도로 이루어진 경로가 나옴
    route = polyline.decode(res['routes'][0]['geometry'])
    
    # 사용할 형식에 맞춰 위경도 좌표의 위치를 바꿔주는 것
    route = list(map(lambda data: [data[1],data[0]] ,route))
    
    return route

In [5]:
# 총 걸리는 시간을 경로의 거리 기준으로 쪼개주는 함수
def extract_timestamp(route, duration):
    
    # 리스트를 numpy이 배열로 변경
    rt = np.array(route)
    # 리스트를 수평 기준으로 합치기
    rt = np.hstack([rt[:-1,:], rt[1:,:]])
    # 각각 직선거리 추출(리스트 형태)
    per = calculate_straight_distance(rt[:,1], rt[:,0], rt[:,3], rt[:,2])
    # 각각의 직선거리를 전체 직선거리의 합으로 나누기
    per = per / np.sum(per)

    # 계산된 비율을 기반으로 각 지점 도착 예상 시간 계산
    timestamp = per * duration
    timestamp = np.hstack([np.array([0]),timestamp])
    timestamp = list(itertools.accumulate(timestamp)) 
    
    return timestamp

In [6]:
# 모든 함수를 한번에 실행하는 코드(trips 데이터의 형태로 저장)
def osrm_routing_machine(O, D):

   # osrm 데이터 생성
   osrm_base, status = get_res([O.x, O.y, D.x, D.y])
   
   # osrm 데이터가 생성 됬으면 진행
   if status == 'defined':
      # 거리 및 걸리는 시간 추출
      duration, distance = extract_duration_distance(osrm_base)
      # 경로 추출
      route = extract_route(osrm_base)
      # timestamp 생성
      timestamp = extract_timestamp(route, duration)
      # 결과 저장
      result = {'route': route, 'timestamp': timestamp, 'duration': duration, 'distance' : distance}
      
      return result
   else: 
      return osrm_base
   
# OD_data 한쌍일 때 osrm_routing_machine작동함수
def osrm_routing_machine_multiprocess(OD):
   O, D = OD
   result = osrm_routing_machine(O, D)
   return result

# OD_data 데이터가 리스트쌍 일때의 osrm_routing_machine 작동함수
def osrm_routing_machine_multiprocess_all(OD_data):
    results = list(map(osrm_routing_machine_multiprocess, OD_data))
    return results

##### [2] OD 데이터 생성

In [10]:
def get_OD_data(point, num=10):
    """
    Generate random origin-destination pairs from given points
    
    Args:
        point (dict): Dictionary containing points data
        num (int): Number of OD pairs to generate
    
    Returns:
        list: List of OD pairs as [Origin Point, Destination Point]
    """
    OD_data = []
    
    # Convert dictionary keys to list for random sampling
    point_keys = list(point.keys())
    
    # Generate num random pairs of coordinates
    for _ in range(num):
        # Generate random pair using the point keys
        neighborhood1, neighborhood2 = rd.sample(point_keys, 2)
        # Get start and end points from the dictionary
        start_point = point[neighborhood1]
        end_point = point[neighborhood2]
        
        # Convert to shapely Point objects
        O = Point(start_point)
        D = Point(end_point)
        # Add origin-destination pair to list
        OD_data.append([O, D])
    
    return OD_data

In [11]:
# 데이터 좌표
point = {
    "가천대_반도체대학" : [127.127384 , 37.450910],
    "가천대_일반대학원" : [127.130112 , 37.452589],
    "가천대_교육대학원" : [127.131698 , 37.452066],
    "가천대_학생회관" : [127.134042 , 37.453336],
    "가천대_ai_공학관" : [127.133374 , 37.455009],
}


In [16]:
# O는 출발지 D는 도착지로 생각하면 편함
OD_data = get_OD_data(point, 30)
OD_data

[[<POINT (127 37.5)>, <POINT (127 37.5)>],
 [<POINT (127 37.5)>, <POINT (127 37.5)>],
 [<POINT (127 37.5)>, <POINT (127 37.5)>],
 [<POINT (127 37.5)>, <POINT (127 37.5)>],
 [<POINT (127 37.5)>, <POINT (127 37.5)>],
 [<POINT (127 37.5)>, <POINT (127 37.5)>],
 [<POINT (127 37.5)>, <POINT (127 37.5)>],
 [<POINT (127 37.5)>, <POINT (127 37.5)>],
 [<POINT (127 37.5)>, <POINT (127 37.5)>],
 [<POINT (127 37.5)>, <POINT (127 37.5)>],
 [<POINT (127 37.5)>, <POINT (127 37.5)>],
 [<POINT (127 37.5)>, <POINT (127 37.5)>],
 [<POINT (127 37.5)>, <POINT (127 37.5)>],
 [<POINT (127 37.5)>, <POINT (127 37.5)>],
 [<POINT (127 37.5)>, <POINT (127 37.5)>],
 [<POINT (127 37.5)>, <POINT (127 37.5)>],
 [<POINT (127 37.5)>, <POINT (127 37.5)>],
 [<POINT (127 37.5)>, <POINT (127 37.5)>],
 [<POINT (127 37.5)>, <POINT (127 37.5)>],
 [<POINT (127 37.5)>, <POINT (127 37.5)>],
 [<POINT (127 37.5)>, <POINT (127 37.5)>],
 [<POINT (127 37.5)>, <POINT (127 37.5)>],
 [<POINT (127 37.5)>, <POINT (127 37.5)>],
 [<POINT (1

##### [3] 통행 데이터 생성

- 출발지, 목적지가 정해졌을 때 그 사이의 경로 및 시간을 생성

In [17]:
# OD, DO 포인트에 대해서 각각의 trips데이터를 생성
OD_results = osrm_routing_machine_multiprocess_all(OD_data)

In [39]:
OD_results[0]['route'], OD_results[0]['timestamp']

([[127.13396, 37.45341],
  [127.13391, 37.45337],
  [127.13375, 37.45324],
  [127.13357, 37.45305],
  [127.13337, 37.45297],
  [127.1332, 37.45291],
  [127.13312, 37.45287],
  [127.13217, 37.45251],
  [127.13208, 37.45247],
  [127.13203, 37.45245],
  [127.13198, 37.45243],
  [127.13192, 37.45239],
  [127.13187, 37.45236],
  [127.13173, 37.45232],
  [127.13176, 37.45229],
  [127.13181, 37.45222]],
 [0.0,
  0.027775816403669218,
  0.11736092526672418,
  0.23454180422035542,
  0.32217336290603144,
  0.3949711028958578,
  0.43196611323988415,
  0.8438756330192221,
  0.8842356547972791,
  0.9061436729522594,
  0.9280516957687719,
  0.9587099632516222,
  0.9832342143871426,
  1.0414555960221352,
  1.0603357309010766,
  1.1])

##### [4] timestamp 변경

- 지금은 모든 통행이 0분에 시작함
- 노이즈를 추가하여 통행시간을 조정

In [29]:
# 원하는 범위에서 랜덤함 숫자를 원하는 만큼 뽑아내는 함수
def sample_interval(start, end, count, num_samples):
    # 시작과 끝을 count만 큼 나눔(최종 : 나눈 만큼의 숫자가 생성됨)
    interval_size = (end - start) / count
    samples = []
    # 랜덤 숫자 생성
    for i in range(count):
        interval_start = start + interval_size * i
        interval_end = interval_start + interval_size
        samples.extend(rd.sample(range(int(interval_start), int(interval_end)), num_samples))
    return samples

### ex.
# 시작 시간과 출발 시간 사이의 랜덤 숫자 생성
# 초 기준
sample_interval(0, 50, 10, 1)

[0, 8, 12, 16, 21, 27, 33, 37, 41, 48]

In [30]:
# 시각화 할때 겹치지 않기 하기 위해서 임의의 시간을 더해주는 함수
# 시간 범위를 정하고 싶으면 아래의  sample_interval의 앞의 두개의 인풋값 바꿔주기!
def timestamp_change(OD_results) :
    random_numbers = sample_interval(0, 50, len(OD_results), 1)
    for i in range(0, len(OD_results)) :
        # (i+1)*3을 더해주는 이유는 출발 시간이 겹치지 않기 하기 위해서
        OD_results[i]['timestamp'] = list(np.array(OD_results[i]['timestamp']) + random_numbers[i])
        
    return OD_results

In [31]:
# timestamp 변경
OD_results = timestamp_change(OD_results)

In [34]:
# 시작 시간이 잘 변경 되었나 확인
OD_results[0]['timestamp'][0], OD_results[-1]['timestamp'][0]

(0.0, 48.0)

##### [5] 데이터 저장

In [35]:
# 데이터 저장
path = '../simulation_base/simulation/public/data/'

with open(os.path.join(path + 'trips.json'), 'w', encoding='utf-8') as file:
    json.dump(OD_results, file)

### 4.2 시스템 아키텍처와 설정

- 목표: React 애플리케이션 설정 및 axios, mapbox-gl 같은 필수 라이브러리 설명
- 구성:
    - 주요 파일 설명 (App.js, index.js)
    - 오프라인 기능을 위한 serviceWorker 역할
    - GitHub에 호스팅된 데이터를 디지털 트윈의 데이터 소스로 사용

#### 4.2.1 Prerequistes
이 절에서는 디지털 트윈 시뮬레이션을 위한 React 애플리케이션의 구조와 설정 방법을 다룬다. 디지털 트윈 시뮬레이션 시스템을 구축하기 위해 필요한 라이브러리 설치, 주요 파일의 역할, 데이터 소스 구성 방식을 하나씩 설명한다. 

```{note}
** 왜 시뮬레이션의 구현을 React에서 했는가?**

예를 들어, 서울의 강남역 부근에서 평일 저녁 6시부터 7시 사이의 택시 수요를 시뮬레이션한다고 가정해보자. 과거 데이터를 분석해 이 시간대에 평균적으로 시간당 30명의 승객이 택시를 잡는다고 가정하자. 택시수요가 포아송 분포를 따르고 평균(λ)을 30으로 설정하면 아래와 같은 결과가 나올 수 있다. 
```

 (설명 추가)

React 애플리케이션을 실행하려면 Node.js가 필요합니다. Node.js는 JavaScript 런타임 환경으로, 서버 측 코드와 웹 애플리케이션 빌드에 널리 사용됩니다.  
Node.js 설치: Node.js 공식 사이트에서 운영체제에 맞는 설치 파일을 다운로드합니다.  
설치가 완료되면 터미널창에서 다음 명령어를 통해 설치가 잘 되었는지 확인할 수 있습니다:  

```bash
node -v
npm -v
```

React에 대한 기본 개념 설명 (1~2문단)



#### 4.2.2 React 애플리케이션 설정

React 애플리케이션은 보통 Create React App을 사용해 간편하게 시작할 수 있다. Create React App은 React 프로젝트의 기본 구조를 만들어주며, 프로젝트를 개발, 테스트, 빌드, 배포하는 데 필요한 다양한 설정을 포함한다.

```bash
npx create-react-app my-simulation-app
```

위 명령어를 실행하면 my-simulation-app이라는 디렉터리에 기본 React 파일 구조가 생성된다. 이 구조는 다음과 같이 구성된다. 

- public/: 공개 폴더로, index.html 파일을 포함하여 웹 애플리케이션의 기본 HTML 템플릿을 제공합니다.
- src/: React 컴포넌트, 스타일 시트, 애플리케이션 로직 등이 포함된 폴더로, 모든 개발이 이곳에서 이루어집니다.
- package.json: 애플리케이션의 메타데이터와 의존성을 정의하는 파일입니다.

### 4.3 데이터 처리 및 컴포넌트 설계

- 목표: 디지털 트윈 시뮬레이션에서 데이터 획득 및 관리 방식을 설명
- 세부 내용:
    - 시뮬레이션 데이터를 가져오기 위한 fetchData 함수
    - 여행 데이터(JSON)의 구조와 역할
    - 여행 데이터를 표시하고 업데이트하는 Trip 컴포넌트의 기능 설명



### 4.4 시뮬레이션 데이터 시각화

- 목표: mapbox-gl을 사용해 지도 기반 데이터를 렌더링하는 방법 설명
- 세부 내용:
    - Mapbox 설정과 지도 스타일링
    - 지도 위에 여행 데이터를 통합하고 데이터 변경을 처리하는 상태 변수 설명
    - 실시간 데이터 시각화 및 업데이트 메커니즘



### 4.5 인터랙티브 기능과 사용자 인터페이스

- 목표: 디지털 트윈 시뮬레이션의 인터랙티브 요소 설명
- 세부 내용:
    - Splash를 사용한 로딩 상태 관리
    - Trip 컴포넌트 내 사용자 상호작용 설명
    - 사용자 경험을 강화하는 방법 (예: 반응형 디자인, 지도 확대/축소 기능 등)


### 4.6 테스트 및 배포

- 목표: 테스트 방법과 애플리케이션 배포 설명
- 세부 내용:
    - 사용 가능한 테스트 프레임워크를 사용한 단위 테스트
    - npm run build를 사용한 프로덕션 빌드
    - React 애플리케이션의 배포 옵션과 모범 사례



### 4.7 사례 연구: 샘플 여행 시뮬레이션

- 목표: 특정 여행 시나리오를 시뮬레이션하며 전체 워크플로우를 데모
- 구성:
    - 샘플 데이터(예: 여행 JSON) 설명
    - GitHub에서 화면 시각화로 데이터 흐름에 대한 단계별 설명
    - 디지털 트윈 결과의 해석 및 실제 적용 사례 설명

### 4.8 Exercise

#### 4.8.1 100개의 od를 뽑아서 시뮬레이션을 만들고 깃허브에 올려서 시뮬레이션 링크를 사이버캠퍼스에 제출

In [ ]:
# 데이터 좌표
point = {
    "중앙시장사거리" : [127.131770, 37.440888],
    "숯골사거리" : [127.142398 , 37.444055],
    "동부센트레빌2단지아파트" : [127.129460 , 37.447540],
    "수진역" : [127.140851 , 37.437443],
    "개별용달" : [127.139292 , 37.446605],
    "버거킹" : [127.150505 , 37.442235],
}

과제 할때 simulation-class/smulation/src/components/Trips.json에 들가면 maxTime 최소 350이상으로 설정하기

longitude = 127.135840
latitude = 37.442836
로 설정하기

- 과제 할때 위의 함수에서 아래와 같이 수정(안하면 시뮬레이션이 잘림)
- random_numbers = sample_interval(0, 50, len(OD_results), 1) ->  random_numbers = sample_interval(0, 300, len(OD_results), 1)

#### 4.8.2 